# Whisper's transcription plus Pyannote's Diarization 

Andrej Karpathy [suggested](https://twitter.com/karpathy/status/1574476200801538048?s=20&t=s5IMMXOYjBI6-91dib6w8g) training a classifier on top of  OpenAI [Whisper](https://openai.com/blog/whisper/) model features to identify the speaker, so we can visualize the speaker in the transcript. But, as [pointed out](https://twitter.com/tarantulae/status/1574493613362388992?s=20&t=s5IMMXOYjBI6-91dib6w8g) by Christian Perone, it seems that features from whisper wouldn't be that great for speaker recognition as its main objective is basically to ignore speaker differences.

In the following, I use [**`pyannote-audio`**](https://github.com/pyannote/pyannote-audio), a speaker diarization toolkit by Hervé Bredin, to identify the speakers, and then match it with the transcriptions of Whispr. I do it on the first 30 minutes of  Lex's 2nd [interview](https://youtu.be/SGzMElJ11Cc) with Yann LeCun. Check the result [**here**](https://majdoddin.github.io/lexicap.html). 

It is tricky to match the transcriptions to diarization segemtns, specially when the speaker changes. To resolve it, Sarah Kaiser [suggested](https://github.com/openai/whisper/discussions/264#discussioncomment-3825375) runnnig the pyannote.audio first and  then just running whisper on the split-by-speaker chunks. 
For sake of performance (and transcription quality?), we attach the audio segements into a single audio file with a silent spacer as a seperator, and run whisper on it. Enjoy it!

# Preparing the audio file

 Installing `yt-dlp` and downloading the [video](https://).

In [2]:
# install from develop branch
!pip install  https://github.com/pyannote/pyannote-audio/archive/refs/heads/develop.zip

  Using cached https://github.com/pyannote/pyannote-audio/archive/refs/heads/develop.zip
  Preparing metadata (setup.py) ... done
  Using cached asteroid_filterbanks-0.4.0-py3-none-any.whl (29 kB)
  Using cached einops-0.6.1-py3-none-any.whl (42 kB)
  Obtaining dependency information for huggingface_hub>=0.13.0 from https://files.pythonhosted.org/packages/7f/c4/adcbe9a696c135578cabcbdd7331332daad4d49b7c43688bc2d36b3a47d2/huggingface_hub-0.16.4-py3-none-any.whl.metadata
  Obtaining dependency information for lightning>=2.0.1 from https://files.pythonhosted.org/packages/a6/a3/e3caca2bb110ad5f33b485d48a6b604d104842b922914c973c0231916131/lightning-2.0.8-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 877.3 kB/s eta 0:00:000:01
  Using cached omegaconf-2.3.0-py3-none-any.whl (79 kB)
  Using cached pyannote.core-5.0.0-py3-none-any.whl (58 kB)
  Using cached pyannote.database-5.0.1-py3-none-any.whl (48 kB)
  Using cached pyannote.metrics-3.2.1-py3-none-any.

In [6]:
!pip install -U yt-dlp

  Obtaining dependency information for yt-dlp from https://files.pythonhosted.org/packages/5c/da/ef08140cea3392288a8f6cd60f6f12510a4c5776eead53e90151f139af19/yt_dlp-2023.7.6-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.0/158.0 kB 1.9 MB/s eta 0:00:00a 0:00:01
  Using cached mutagen-1.46.0-py3-none-any.whl (193 kB)
  Obtaining dependency information for pycryptodomex from https://files.pythonhosted.org/packages/45/09/ce11344724e674957402b5f3c0d663388f755ecd1c864f0c405b5f959bd8/pycryptodomex-3.18.0-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached Brotli-1.0.9-cp38-cp38-manylinux1_x86_64.whl (357 kB)
Using cached yt_dlp-2023.7.6-py2.py3-none-any.whl (3.0 MB)
Using cached pycryptodomex-3.18.0-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)


In [4]:
!wget -O - -q  https://github.com/yt-dlp/FFmpeg-Builds/releases/download/latest/ffmpeg-master-latest-linux64-gpl.tar.xz | xz -qdc| tar -x

In [1]:
from tempfile import NamedTemporaryFile


In [2]:
import subprocess

def convertFileFromWebOrLocalToAudio(url, output):
    subprocess.run(['yt-dlp',
                    '--enable-file-urls',
                    '--force-overwrites', 
                    '-xv', '--ffmpeg-location',  
                    'ffmpeg-master-latest-linux64-gpl/bin',  
                    '--audio-format',  
                    'wav',   
                    '-o',  
                    output,  
                    '--',  
                    url])


In [3]:
temp = NamedTemporaryFile(suffix='.wav', dir='./temp/')
tempCut = NamedTemporaryFile(suffix='.wav', dir='./temp/')
tempSegmented = NamedTemporaryFile(suffix='.wav', dir='./temp/')




In [64]:
print(temp.name)

/home/gabriel_vr/Desktop/work/fator3/startup_manager/whisper-api/temp/tmpv6p3r266.wav


In [4]:
convertFileFromWebOrLocalToAudio('file:///home/gabriel_vr/Downloads/O Maior Segredo da Nilce! - Hora de Pôr Café (Parte 67).mp4', temp.name)


[debug] Command-line config: ['--enable-file-urls', '--force-overwrites', '-xv', '--ffmpeg-location', 'ffmpeg-master-latest-linux64-gpl/bin', '--audio-format', 'wav', '-o', '/home/gabriel_vr/Desktop/work/fator3/startup_manager/whisper-api/temp/tmpu5ndqx45.wav', '--', 'file:///home/gabriel_vr/Downloads/O Maior Segredo da Nilce! - Hora de Pôr Café (Parte 67).mp4']
[debug] Encodings: locale UTF-8, fs utf-8, pref UTF-8, out utf-8 (No ANSI), error utf-8 (No ANSI), screen utf-8 (No ANSI)
[debug] yt-dlp version stable@2023.07.06 [b532a3481] (pip)
[debug] Python 3.8.17 (CPython x86_64 64bit) - Linux-5.14.0-1059-oem-x86_64-with-glibc2.10 (OpenSSL 3.1.2 1 Aug 2023, glibc 2.31)
[debug] exe versions: ffmpeg N-111869-g7aa71ab5c0-20230831 (setts), ffprobe N-111869-g7aa71ab5c0-20230831
[debug] Optional libraries: Cryptodome-3.18.0, brotli-1.0.9, certifi-2023.07.22, mutagen-1.46.0, sqlite3-2.6.0, websockets-11.0.3
[debug] Proxy map: {}
[debug] Loaded 1855 extractors


[generic] Extracting URL: file:///home/gabriel_vr/Downloads/O Maior Segredo da Nilce! - Hora de Pôr Café (Parte 67).mp4
[generic] O Maior Segredo da Nilce! - Hora de Pôr Café (Parte 67): Downloading webpage
[redirect] Following redirect to file:///home/gabriel_vr/Downloads/O%20Maior%20Segredo%20da%20Nilce!%20-%20Hora%20de%20P%C3%B4r%20Caf%C3%A9%20(Parte%2067).mp4
[generic] Extracting URL: file:///home/gabriel_vr/Downloads/O%20Maior%20Segredo%20da%20Nilce!%20-%20Hora%20de%20P%C3%B4r%20Caf%C3%A9%20(Parte%2067).mp4
[generic] O Maior Segredo da Nilce! - Hora de Pôr Café (Parte 67): Downloading webpage
[info] O Maior Segredo da Nilce! - Hora de Pôr Café (Parte 67): Downloading 1 format(s): mp4
Deleting existing file /home/gabriel_vr/Desktop/work/fator3/startup_manager/whisper-api/temp/tmpu5ndqx45.wav
[download] Destination: /home/gabriel_vr/Desktop/work/fator3/startup_manager/whisper-api/temp/tmpu5ndqx45.mp4
[download] 100% of   43.88MiB in 00:00:00 at 509.56MiB/s   
[ExtractAudio] Destinat

[debug] Identified a direct video link
[debug] Formats sorted by: hasvid, ie_pref, lang, quality, res, fps, hdr:12(7), vcodec:vp9.2(10), channels, acodec, size, br, asr, proto, vext, aext, hasaud, source, id
[debug] Invoking http downloader on "file:///home/gabriel_vr/Downloads/O%20Maior%20Segredo%20da%20Nilce!%20-%20Hora%20de%20P%C3%B4r%20Caf%C3%A9%20(Parte%2067).mp4"
[debug] ffmpeg command line: ffmpeg-master-latest-linux64-gpl/bin/ffprobe -show_streams file:/home/gabriel_vr/Desktop/work/fator3/startup_manager/whisper-api/temp/tmpu5ndqx45.mp4
[debug] ffmpeg command line: ffmpeg-master-latest-linux64-gpl/bin/ffmpeg -y -loglevel repeat+info -i file:/home/gabriel_vr/Desktop/work/fator3/startup_manager/whisper-api/temp/tmpu5ndqx45.mp4 -vn -movflags +faststart file:/home/gabriel_vr/Desktop/work/fator3/startup_manager/whisper-api/temp/tmpu5ndqx45.wav


Deleting original file /home/gabriel_vr/Desktop/work/fator3/startup_manager/whisper-api/temp/tmpu5ndqx45.mp4 (pass -k to keep)



Cutting the first 20 minutes of the video for further process.


In [7]:
!pip install pydub

  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)


In [5]:
from pydub import AudioSegment


def cutAudio(path, pathDest, begin, end):
    newAudio = AudioSegment.from_wav(path)
    a = newAudio[begin:end]
    a.export(pathDest, format="wav") 


In [6]:
t1 = 0 * 1000 #Works in milliseconds
t2 = 1 * 60 * 1000
cutAudio(temp.name, tempCut.name, t1, t2)

`pyannote.audio` seems to miss the first 0.5 seconds of the audio, and, therefore, we prepend a spcacer.

In [7]:
from pydub import AudioSegment
def prependToAudio(path,time):
    audio = AudioSegment.from_wav(path)
    spacer = AudioSegment.silent(duration=time)
    audio = spacer.append(audio, crossfade=0)
    audio.export(path, format='wav')



In [8]:
spacermilli = 2500
prependToAudio(tempCut,spacermilli)

# Pyannote's Diarization

[`pyannote.audio`](https://github.com/pyannote/pyannote-audio) is an open-source toolkit written in Python for **speaker diarization**. 

Based on [`PyTorch`](https://pytorch.org) machine learning framework, it provides a set of trainable end-to-end neural building blocks that can be combined and jointly optimized to build speaker diarization pipelines. 

`pyannote.audio` also comes with pretrained [models](https://huggingface.co/models?other=pyannote-audio-model) and [pipelines](https://huggingface.co/models?other=pyannote-audio-pipeline) covering a wide range of domains for voice activity detection, speaker segmentation, overlapped speech detection, speaker embedding reaching state-of-the-art performance for most of them. 

Installing Pyannote and running it on the video to generate the diarizations.

In [3]:
!pip install   pyannote.audio

  Using cached torchaudio-0.13.1-cp310-cp310-manylinux1_x86_64.whl (4.2 MB)
  Using cached networkx-2.8.8-py3-none-any.whl (2.0 MB)
  Using cached torch-1.13.1-cp310-cp310-manylinux1_x86_64.whl (887.5 MB)
^C
ERROR: Operation cancelled by user


In [9]:
from pyannote.audio import Pipeline

def diarizeAudio(path):
    pipeline = Pipeline.from_pretrained('pyannote/speaker-diarization', use_auth_token='')
    dz = pipeline(path)  
    return dz

/home/gabriel_vr/Desktop/work/fator3/startup_manager/whisper-api/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
diarization = diarizeAudio(tempCut.name)

torchvision is not available - cannot save figures
Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.0.8. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../../../../.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 2.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.0.1+cu117. Bad things might happen unless you revert torch to 1.x.


In [20]:
print(*list(diarization.itertracks(yield_label = True))[:10], sep="\n")

(<Segment(3.02901, 4.49659)>, 'G', 'SPEAKER_01')
(<Segment(4.88908, 10.9983)>, 'A', 'SPEAKER_00')
(<Segment(11.8686, 17.9949)>, 'H', 'SPEAKER_01')
(<Segment(17.9949, 31.3225)>, 'B', 'SPEAKER_00')
(<Segment(31.3225, 32.9949)>, 'I', 'SPEAKER_01')
(<Segment(33.5239, 35.4693)>, 'C', 'SPEAKER_00')
(<Segment(36.4761, 42.0051)>, 'J', 'SPEAKER_01')
(<Segment(42.0051, 47.6024)>, 'D', 'SPEAKER_00')
(<Segment(49.4283, 50.9983)>, 'E', 'SPEAKER_00')
(<Segment(50.9983, 56.3396)>, 'K', 'SPEAKER_01')


In [74]:
list(diarization.itertracks(yield_label = True))[0][0].start

2.5511945392491473

In [75]:
def millisec(timeStr):
  spl = timeStr.split(":")
  s = (int)((int(spl[0]) * 60 * 60 + int(spl[1]) * 60 + float(spl[2]) )* 1000)
  return s

In [18]:
def getSpeeches(diarization, spacer=0.0):
    speeches = []
    for speech in list(diarization.itertracks(yield_label = True)):
        start = round(speech[0].start, 3)
        end = round(speech[0].end, 3)
        speaker = speech[2]
        speeches.append({
            "start": start,
            "end": end,
            "speaker": speaker
        })
    return speeches

In [19]:
speeches = getSpeeches(diarization, spacermilli)

In [21]:
print(speeches)

[{'start': 3.029, 'end': 4.497, 'speaker': 'SPEAKER_01'}, {'start': 4.889, 'end': 10.998, 'speaker': 'SPEAKER_00'}, {'start': 11.869, 'end': 17.995, 'speaker': 'SPEAKER_01'}, {'start': 17.995, 'end': 31.323, 'speaker': 'SPEAKER_00'}, {'start': 31.323, 'end': 32.995, 'speaker': 'SPEAKER_01'}, {'start': 33.524, 'end': 35.469, 'speaker': 'SPEAKER_00'}, {'start': 36.476, 'end': 42.005, 'speaker': 'SPEAKER_01'}, {'start': 42.005, 'end': 47.602, 'speaker': 'SPEAKER_00'}, {'start': 49.428, 'end': 50.998, 'speaker': 'SPEAKER_00'}, {'start': 50.998, 'end': 56.34, 'speaker': 'SPEAKER_01'}, {'start': 58.746, 'end': 62.5, 'speaker': 'SPEAKER_00'}, {'start': 59.138, 'end': 60.862, 'speaker': 'SPEAKER_01'}]


In [42]:
import re
dz = open('diarization.txt').read().splitlines()
dzList = []
for l in dz:
  start, end =  tuple(re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=l))
  print(millisec(start), millisec(end))
  start = millisec(start) - spacermilli
  end = millisec(end)  - spacermilli
  speaker = re.findall('SPEAKER_\d+', string=l)
  if(len(speaker) == 0):
    speaker = ['Speaker not identified']
  dzList.append([start, end, speaker[0]])

print(*dzList[:10], sep='\n')

2551 4001
4377 10503
11373 17005
[551, 2001, 'SPEAKER_02']
[2377, 8503, 'SPEAKER_00']
[9373, 15005, 'SPEAKER_01']


# Preparing audio file from the diarization

Attaching audio segements according to the diarization, with a spacer as the delimiter.

In [22]:
def addTimeBetweenSpeeches(path, pathOutput, time, speeches):
    segments = []
    spacer = AudioSegment.silent(duration=time)
    sounds = spacer
    audio = AudioSegment.from_wav(path)

    for speech in speeches:
        start = speech["start"]
        end =  speech["end"]
        
        segments.append(len(sounds))
        sounds = sounds.append(audio[start*1000:end*1000], crossfade=0)
        sounds = sounds.append(spacer, crossfade=0)

        sounds.export(pathOutput, format="wav") #Exports to a wav file in the current path.
    return segments



In [23]:
segments = addTimeBetweenSpeeches(tempCut.name, tempSegmented.name, spacermilli, speeches)

In [24]:
segments[:8]

[2500, 6468, 15077, 23703, 39530, 43702, 48147, 56176]

Freeing up some memory

# Whisper's Transcriptions

Installing Open AI whisper.

**Important:** There is a version conflict with pyannote.audio resulting in an error (see this RP). Our workaround is to first run Pyannote and then whisper. You can safely ignore the error.


In [21]:
!pip install git+https://github.com/openai/whisper.git 

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-tjegbnxt
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-tjegbnxt
  Resolved https://github.com/openai/whisper.git to commit e8622f9afc4eba139bf796c210f5c01081000472
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for more-itertools from https://files.pythonhosted.org/packages/5a/cb/6dce742ea14e47d6f565589e859ad225f2a5de576d7696e0623b784e226b/more_itertools-10.1.0-py3-none-any.whl.metadata
  Using cached tiktoken-0.3.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.7 MB)
  Obtaining dependency information for regex>=2022.1.18 from https://files.pythonhosted.org/packages/1f/5c/374ac3fa3c7ed9a967ad273a5e841897ef6b10aa6aad938ff10717a3e2a3/regex-2023.8.8-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━

Running Open AI whisper on the prepared audio file. [link text](https://) It writes the transcription into a file.

In [43]:
import whisper
def transcribeAudio(path):
    model = whisper.load_model('large')
    result = model.transcribe(path)
    return result['segments']


In [44]:
whisper.available_models()

['tiny.en',
 'tiny',
 'base.en',
 'base',
 'small.en',
 'small',
 'medium.en',
 'medium',
 'large-v1',
 'large-v2',
 'large']

In [45]:
transcription = transcribeAudio(tempSegmented.name)

In [46]:
import numpy as np
print(np.array(segments) - spacermilli)

[    0  3968 12577 21203 37030 41202 45647 53676 61773 65843 73685 79939]


In [47]:
print(transcription)

[{'id': 0, 'seek': 0, 'start': 0.0, 'end': 2.0, 'text': ' Senhoras e senhores', 'tokens': [50364, 43792, 296, 308, 3151, 39356, 50464], 'temperature': 0.0, 'avg_logprob': -0.3690617991759714, 'compression_ratio': 2.5522388059701493, 'no_speech_prob': 0.6564409732818604}, {'id': 1, 'seek': 0, 'start': 2.0, 'end': 4.0, 'text': ' Senhoras e senhores', 'tokens': [50464, 43792, 296, 308, 3151, 39356, 50564], 'temperature': 0.0, 'avg_logprob': -0.3690617991759714, 'compression_ratio': 2.5522388059701493, 'no_speech_prob': 0.6564409732818604}, {'id': 2, 'seek': 0, 'start': 4.0, 'end': 6.0, 'text': ' Essa é a hora', 'tokens': [50564, 22818, 1136, 257, 15098, 50664], 'temperature': 0.0, 'avg_logprob': -0.3690617991759714, 'compression_ratio': 2.5522388059701493, 'no_speech_prob': 0.6564409732818604}, {'id': 3, 'seek': 0, 'start': 6.0, 'end': 8.0, 'text': ' A hora que você quer', 'tokens': [50664, 316, 15098, 631, 2723, 7083, 50764], 'temperature': 0.0, 'avg_logprob': -0.3690617991759714, 'compr

In [49]:
import math
def attachTranscriptionToSpeech(transcription, speeches, segments):
    segmentIdx = 0
    transcribedSegmentIdx = 0
    for segmentIdx in range(len(segments)):
        #startTime = segments[segmentIdx] - spacermilli
        endTime = segments[segmentIdx + 1] - spacermilli if  segmentIdx < len(segments) - 1 else math.inf
        text = []
        #print(int(transcription[transcribedSegmentIdx]['start'] * 1000), int(transcription[transcribedSegmentIdx]['end'] * 1000) )
        while(transcribedSegmentIdx < len(transcription) and int(transcription[transcribedSegmentIdx]['start'] * 1000) <= endTime):
            text.append(transcription[transcribedSegmentIdx]['text'])
            transcribedSegmentIdx+=1
        speeches[segmentIdx]['text'] = "".join(text)

In [50]:
attachTranscriptionToSpeech(transcription, speeches, segments)

In [51]:
print(speeches)

[{'start': 3.029, 'end': 4.497, 'speaker': 'SPEAKER_01', 'text': ' Senhoras e senhores Senhoras e senhores'}, {'start': 4.889, 'end': 10.998, 'speaker': 'SPEAKER_00', 'text': ' Essa é a hora A hora que você quer Essa é a hora A hora de porra Uma categoria que vocês sugeriram'}, {'start': 11.869, 'end': 17.995, 'speaker': 'SPEAKER_01', 'text': ' Piores nomes pra se dar pra bebês Nascidos durante a pandemia Nascidos durante a pandemia Nascidos durante a pandemia'}, {'start': 17.995, 'end': 31.323, 'speaker': 'SPEAKER_00', 'text': ' Nascido durante a pandemia Nascido durante a pandemia Nascido durante a pandemia Pandemila Pandemila José Bento Virulento Mascarine'}, {'start': 31.323, 'end': 32.995, 'speaker': 'SPEAKER_01', 'text': ' Mascarine Mascarine'}, {'start': 33.524, 'end': 35.469, 'speaker': 'SPEAKER_00', 'text': ' Nossa agora que eu entendi Demorei um pouquinho'}, {'start': 36.476, 'end': 42.005, 'speaker': 'SPEAKER_01', 'text': ' Tem um que mandou um monte E a Ederson Papelson e J

In [53]:
import json
f = open('speech.json','w')
f.write(json.dumps(speeches))

1793

In [29]:
print(speeches)

[{'start': 3.524, 'end': 4.991, 'speaker': 'SPEAKER_00', 'text': ' Muitas vezes eu não consigo me sentir bem. Senhoras e senhores.'}, {'start': 5.401, 'end': 11.493, 'speaker': 'SPEAKER_01', 'text': ' Essa é a hora, a hora que você quer. Essa é a hora, a hora de...'}, {'start': 12.346, 'end': 17.995, 'speaker': 'SPEAKER_00', 'text': ' Uma categoria que vocês sugeriram, piores nomes pra se dar pra bebês nascidos durante a pandemia.'}, {'start': 17.995, 'end': 31.493, 'speaker': 'SPEAKER_02', 'text': ' Deve ter nascido muitos, né? Porque o povo ficou em casa. Piores nomes. Pandemila. Pandemila. José Bento virulento.'}, {'start': 31.493, 'end': 33.2, 'speaker': 'SPEAKER_00', 'text': ' Mascarini.'}, {'start': 34.036, 'end': 35.725, 'speaker': 'SPEAKER_02', 'text': ' Nossa, agora eu entendi. Demorou.'}, {'start': 36.971, 'end': 42.739, 'speaker': 'SPEAKER_00', 'text': ' Tem um que mandou um monte. E a Ederson? Papelson Higienison. Alcolino 19.'}, {'start': 42.739, 'end': 48.183, 'speaker': 

Reading the transcription file.

In [24]:
!pip install -U webvtt-py

In [31]:
import webvtt

captions = [[(int)(millisec(caption.start)), (int)(millisec(caption.end)),  caption.text] for caption in webvtt.read('dz.vtt')]
print(*captions[:8], sep='\n')

[0, 5320, 'e aí senhoras e senhores']
[5320, 14760, 'essa é a hora a hora que você quer essa é a hora hora de uma categoria que']
[14760, 19640, 'vocês sugeriram piores nomes para se dar para bebês nascidos durante a']


In [40]:
print(*dzList, sep='\n')

[551, 2001, 'SPEAKER_02']
[2377, 8503, 'SPEAKER_00']
[9373, 15005, 'SPEAKER_01']


# Matching the Transcriptions and the Diarizations

Matching each trainscrition line to some diarizations, and generating the HTML file. To get the correct timing, we should take care of the parts in original audio that were in no diarization segment.

In [27]:
preS = '<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <meta http-equiv="X-UA-Compatible" content="ie=edge">\n    <title>Lexicap</title>\n    <style>\n        body {\n            font-family: sans-serif;\n            font-size: 18px;\n            color: #111;\n            padding: 0 0 1em 0;\n        }\n        .l {\n          color: #050;\n        }\n        .s {\n            display: inline-block;\n        }\n        .e {\n            display: inline-block;\n        }\n        .t {\n            display: inline-block;\n        }\n        #player {\n\t\tposition: sticky;\n\t\ttop: 20px;\n\t\tfloat: right;\n\t}\n    </style>\n  </head>\n  <body>\n    <h2>Yann LeCun: Dark Matter of Intelligence and Self-Supervised Learning | Lex Fridman Podcast #258</h2>\n  <div  id="player"></div>\n    <script>\n      var tag = document.createElement(\'script\');\n      tag.src = "https://www.youtube.com/iframe_api";\n      var firstScriptTag = document.getElementsByTagName(\'script\')[0];\n      firstScriptTag.parentNode.insertBefore(tag, firstScriptTag);\n      var player;\n      function onYouTubeIframeAPIReady() {\n        player = new YT.Player(\'player\', {\n          height: \'210\',\n          width: \'340\',\n          videoId: \'SGzMElJ11Cc\',\n        });\n      }\n      function setCurrentTime(timepoint) {\n        player.seekTo(timepoint);\n   player.playVideo();\n   }\n    </script><br>\n'
postS = '\t</body>\n</html>'

In [32]:
print(segments)

[2000, 5450, 13576]


In [49]:
from datetime import timedelta

html = list(preS)

for i in range(len(segments)):
  idx = 0
  #print(segments[i])
  for idx in range(len(captions)):
    #print(captions[idx])
    if captions[idx][0] >= (segments[i] - spacermilli):
      break;
  
  while (idx < (len(captions))) and ((i == len(segments) - 1) or (captions[idx][1] < segments[i+1])):
    c = captions[idx]  
    print('segment', segments[i])
    print('caption', c)
    print('diarization', dzList[i])
    
    start = dzList[i][0] + (c[0] -segments[i])
    end = dzList[i][1] + (c[1] -segments[i])

    if start < 0: 
      start = 0
    idx += 1

    start = start / 1000.0
    startStr = '{0:02d}:{1:02d}:{2:02.2f}'.format((int)(start // 3600), 
                                            (int)(start % 3600 // 60), 
                                            start % 60)
    endStr = '{0:02d}:{1:02d}:{2:02.2f}'.format((int)(end // 3600), 
                                            (int)(end % 3600 // 60), 
                                            end % 60)
    
    print('{} {} - {}: {}'.format(dz[i][2], startStr, endStr, c[2]))


segment 2000
caption [0, 5320, 'e aí senhoras e senhores']
diarization [551, 2001, 'SPEAKER_02']
0 00:00:0.00 - 01:28:41.00: e aí senhoras e senhores
segment 13576
caption [14760, 19640, 'vocês sugeriram piores nomes para se dar para bebês nascidos durante a']
diarization [9373, 15005, 'SPEAKER_01']
0 00:00:10.56 - 05:51:9.00: vocês sugeriram piores nomes para se dar para bebês nascidos durante a


In [50]:
print(segments)

[2000, 5450, 13576]
